In [1]:
import os, subprocess
from datetime import datetime
from resource import getrusage as resource_usage, RUSAGE_CHILDREN
from time import time as timestamp
import json
import statistics


In [2]:
class Tester:
    def __init__(self, name, fuzzer_executable, grammar_file, depths, iterations=2, timeout=3600):
        """
        Initializes the Tester instance.

        :param name: Name of the test (combination of fuzzer and grammar)
        :param fuzzer_executable: Executable file of the fuzzer compiler
        :param grammar_file: Path to the grammar file
        :param depths: List of depths to test
        :param iterations: Number of iterations per depth
        :param timeout: Timeout for each test run
        """
        self.name = name
        self.fuzzer_executable = fuzzer_executable
        self.grammar_file = grammar_file
        self.depths = depths
        self.iterations = iterations
        self.timeout = timeout
        self.results = {}

    def generate_c_code(self, max_depth):
        """
        Generates the C code from the grammar file at the specified depth.

        :param max_depth: Maximum depth for the fuzzer
        :return: Path to the generated C file
        """
        output_c = f"generated/{self.name}_{max_depth}.c"
        cmd = f"./{self.fuzzer_executable} -d {max_depth} -p {self.grammar_file} -o {output_c}"
        if not os.path.exists(output_c):
            print(f"Generating C code for depth {max_depth}...")
            subprocess.run(cmd, shell=True, check=True)
        else:
            print(f"C code for depth {max_depth} already exists. Skipping generation.")
        return output_c

    def compile_c_code(self, output_c):
        """
        Compiles the generated C code into an executable using clang.

        :param output_c: Path to the generated C file
        :return: Name of the compiled executable
        """
        executable = output_c.replace('.c', '')
        if not os.path.exists(executable):
            print(f"Compiling {output_c}...")
            compile_cmd = f"gcc-14 -I/Library/Developer/CommandLineTools/SDKs/MacOSX.sdk/usr/include {output_c} -O2 -o {executable}"
            subprocess.run(compile_cmd, shell=True, check=True)
        else:
            print(f"Executable {executable} already exists. Skipping compilation.")
        return executable

    def run_executable(self, executable):
        """
        Runs the compiled executable and measures its execution time.

        :param executable: Executable file generated from the C code
        :return: Execution time in seconds
        """
        start_time = datetime.now()
        subprocess.run(f"./{executable}", shell=True, check=True, timeout=self.timeout)
        end_time = datetime.now()
        execution_time = (end_time - start_time).total_seconds()
        return execution_time

    def run_test(self):
        """
        Runs tests for each specified depth and iteration, collecting results.
        """
        # Ensure the generated code directory exists
        os.makedirs('generated', exist_ok=True)

        for depth in self.depths:
            print(f"\nTesting {self.name} at depth {depth}")
            depth_results = []
            # Generate C code and compile it once per depth
            output_c = self.generate_c_code(depth)
            executable = self.compile_c_code(output_c)

            for seed in range(self.iterations):
                print(f"Iteration {seed + 1}/{self.iterations}")
                # Run the executable
                try:
                    execution_time = self.run_executable(executable)
                except subprocess.TimeoutExpired:
                    print(f"Execution timed out for {executable}")
                    execution_time = self.timeout
                # Collect results
                output_size = os.path.getsize('output.txt') if os.path.exists('output.txt') else 0
                depth_results.append({'execution_time': execution_time, 'output_size': output_size})
                # Clean up output file
                if os.path.exists('output.txt'):
                    os.remove('output.txt')
            # Calculate average results for the depth
            avg_time = statistics.mean([res['execution_time'] for res in depth_results])
            avg_size = statistics.mean([res['output_size'] for res in depth_results])
            self.results[depth] = {'average_time': avg_time, 'average_size': avg_size}
            print(f"Depth {depth}: Average Time = {avg_time:.2f}s, Average Size = {avg_size / 1024:.2f}KB")
            # Optional: Clean up the generated executable and C code after testing
            # os.remove(executable)
            # os.remove(output_c)

    def save_results(self):
        """
        Saves the collected results into a JSON file.
        """
        os.makedirs('results', exist_ok=True)
        result_file = f'results/{self.name}_results.json'
        with open(result_file, 'w') as f:
            json.dump(self.results, f, indent=4)
        print(f"Results saved to {result_file}")

In [3]:
def run():
    # List of fuzzers and their source files
    fuzzers = [
        {'name': 'DT', 'source': 'DT.cpp'},
        {'name': 'IDT', 'source': 'IDT.cpp'},
        {'name': 'baresubroutine', 'source': 'baresubroutine.cpp'},
        {'name': 'subroutine', 'source': 'subroutine.cpp'},
        # {'name': 'switch', 'source': 'switch.cpp'},
        {'name': 'context', 'source': 'context.cpp'}
    ]

    # List of grammar files
    grammar_files = [
        'grammars/control_flow.json',
        'grammars/css.json',
        'grammars/html.json',
        'grammars/if-else.json',
        'grammars/json.json',
        'grammars/math.json',
        'grammars/programming.json',
        'grammars/query.json',
        'grammars/recursive.json',
        'grammars/regular_expression.json'
    ]

    # Depths to test
    depths = [20,25,30,35,40,45,50,55,60]

    # Compile the fuzzer source files using clang++
    for fuzzer in fuzzers:
        fuzzer_executable = f"{fuzzer['name']}_compiler"
        compile_cmd = f"clang++ {fuzzer['source']} -o {fuzzer_executable}"
        print(f"Compiling {fuzzer['source']} to {fuzzer_executable}...")
        subprocess.run(compile_cmd, shell=True, check=True)
        fuzzer['executable'] = fuzzer_executable

    # Run tests for each fuzzer and grammar combination
    for grammar_file in grammar_files:
        print(f"\nTesting with grammar file: {grammar_file}")
        for fuzzer in fuzzers:
            tester = Tester(
                name=f"{fuzzer['name']}_{os.path.basename(grammar_file).replace('.json', '')}",
                fuzzer_executable=fuzzer['executable'],
                grammar_file=grammar_file,
                depths=depths,
                iterations=20  # Adjust the number of iterations as needed
            )
            tester.run_test()
            tester.save_results()

In [4]:
run()

Compiling DT.cpp to DT_compiler...
Compiling IDT.cpp to IDT_compiler...
Compiling baresubroutine.cpp to baresubroutine_compiler...
Compiling subroutine.cpp to subroutine_compiler...
Compiling context.cpp to context_compiler...

Testing with grammar file: grammars/control_flow.json

Testing DT_control_flow at depth 20
Generating C code for depth 20...
Code written to file successfully.
Compiling generated/DT_control_flow_20.c...
Iteration 1/20
Iteration 2/20
Iteration 3/20
Iteration 4/20
Iteration 5/20
Iteration 6/20
Iteration 7/20
Iteration 8/20
Iteration 9/20
Iteration 10/20
Iteration 11/20
Iteration 12/20
Iteration 13/20
Iteration 14/20
Iteration 15/20
Iteration 16/20
Iteration 17/20
Iteration 18/20
Iteration 19/20
Iteration 20/20
Depth 20: Average Time = 0.02s, Average Size = 1.16KB

Testing DT_control_flow at depth 25
Generating C code for depth 25...
Code written to file successfully.
Compiling generated/DT_control_flow_25.c...
Iteration 1/20
Iteration 2/20
Iteration 3/20
Iteratio

In [5]:
import os
import json
import matplotlib.pyplot as plt
import numpy as np

In [6]:
def read_and_organize_results(results_dir='results'):
    """
    Reads JSON result files and organizes data for comparative plotting.

    :param results_dir: Directory containing result JSON files.
    :return: A dictionary organized by grammar, then by metric, containing compilers' data over depths.
    """
    data = {}

    # List all JSON files in the results directory
    for filename in os.listdir(results_dir):
        if filename.endswith('_results.json'):
            filepath = os.path.join(results_dir, filename)
            with open(filepath, 'r') as f:
                result = json.load(f)

            # Extract fuzzer name and grammar name from the filename
            basename = filename.replace('_results.json', '')
            parts = basename.split('_')
            fuzzer_name = parts[0]
            grammar_name = '_'.join(parts[1:])

            # Initialize nested dictionaries
            if grammar_name not in data:
                data[grammar_name] = {'throughputs': {}, 'times': {}, 'sizes': {}}

            # Iterate over depths and collect data
            for depth_str, metrics in result.items():
                depth = int(depth_str)
                avg_time = metrics['average_time']
                avg_size = metrics['average_size']  # in bytes

                # Calculate throughput (KB/s)
                throughput = (avg_size / 1024) / avg_time if avg_time > 0 else 0

                # Initialize depth data structures if necessary
                if depth not in data[grammar_name]['throughputs']:
                    data[grammar_name]['throughputs'][depth] = {}
                    data[grammar_name]['times'][depth] = {}
                    data[grammar_name]['sizes'][depth] = {}

                # Store the data
                data[grammar_name]['throughputs'][depth][fuzzer_name] = throughput
                data[grammar_name]['times'][depth][fuzzer_name] = avg_time
                data[grammar_name]['sizes'][depth][fuzzer_name] = avg_size / 1024  # Convert to KB

    return data

In [7]:
def plot_comparative_metrics(data, metric_name='throughputs', ylabel='Throughput (KB/s)', output_dir='comparative_plots'):
    """
    Plots comparative metrics for different compilers on the same graph, grouped by grammar.

    :param data: The organized data dictionary from read_and_organize_results().
    :param metric_name: The metric to plot ('throughputs', 'times', 'sizes').
    :param ylabel: Label for the Y-axis.
    :param output_dir: Directory where the plots will be saved.
    """
    os.makedirs(output_dir, exist_ok=True)

    for grammar_name, metrics in data.items():
        depths = sorted(metrics[metric_name].keys())
        compilers = set()

        # Collect all compiler names
        for depth in depths:
            compilers.update(metrics[metric_name][depth].keys())

        compilers = sorted(compilers)

        plt.figure()
        for compiler in compilers:
            values = []
            for depth in depths:
                value = metrics[metric_name][depth].get(compiler, None)
                if value is not None:
                    values.append(value)
                else:
                    values.append(np.nan)  # Handle missing data

            plt.plot(depths, values, marker='o', label=compiler)

        plt.title(f'Comparison of {metric_name.capitalize()} - Grammar: {grammar_name}')
        plt.xlabel('Depth')
        plt.ylabel(ylabel)
        plt.legend()
        plt.grid(True)

        # Save the plot
        plot_filename = f'{grammar_name}_{metric_name}.png'
        plt.savefig(os.path.join(output_dir, plot_filename))
        plt.close()
        print(f'Plot saved: {os.path.join(output_dir, plot_filename)}')

In [8]:
def draw():
    # Step 1: Read and organize the results
    data = read_and_organize_results('results')

    # Step 2: Plot comparative throughput
    plot_comparative_metrics(data, metric_name='throughputs', ylabel='Throughput (KB/s)', output_dir='comparative_plots/throughput')

    # Step 3: Plot comparative execution times
    plot_comparative_metrics(data, metric_name='times', ylabel='Average Execution Time (s)', output_dir='comparative_plots/time')

    # Step 4: Plot comparative output sizes
    plot_comparative_metrics(data, metric_name='sizes', ylabel='Average Output Size (KB)', output_dir='comparative_plots/size')

In [9]:
draw()

Plot saved: comparative_plots/throughput/math_throughputs.png
Plot saved: comparative_plots/throughput/if-else_throughputs.png
Plot saved: comparative_plots/throughput/json_throughputs.png
Plot saved: comparative_plots/throughput/query_throughputs.png
Plot saved: comparative_plots/throughput/recursive_throughputs.png
Plot saved: comparative_plots/throughput/control_flow_throughputs.png
Plot saved: comparative_plots/throughput/css_throughputs.png
Plot saved: comparative_plots/throughput/html_throughputs.png
Plot saved: comparative_plots/throughput/regular_expression_throughputs.png
Plot saved: comparative_plots/throughput/programming_throughputs.png
Plot saved: comparative_plots/time/math_times.png
Plot saved: comparative_plots/time/if-else_times.png
Plot saved: comparative_plots/time/json_times.png
Plot saved: comparative_plots/time/query_times.png
Plot saved: comparative_plots/time/recursive_times.png
Plot saved: comparative_plots/time/control_flow_times.png
Plot saved: comparative_pl